In [4]:
import pandas as pd
import numpy as np
import ROOT

data = pd.read_csv("/eos/user/w/wrsmith/SWAN_projects/NEW_March/SettingUp/DOF_P_Jun18_5_1_20Nov_with_stripID_NEW.txt",  sep=r'\t', engine='python')

EEpos = data[(data.CMSSWID > 872000000) & (data.eta > 0)]
EEneg = data[(data.CMSSWID > 872000000) & (data.eta < 0)]

#Decimal weights to encoded weights8
def decimal_to_encoded(weights):
    enc_weights = [0]*5
    count = -1
    for w in weights:
        count = count+1
        if w >0:
            encodedw = w*64
            encodedw = round(encodedw, 0)
            encodedw = int(encodedw)
            enc_weights[count] = encodedw
        else:
            encodedw = w*64
            encodedw = abs(encodedw)
            encodedw = round(encodedw, 0)
            encodedw = 128 - encodedw
            encodedw = int(encodedw)
            enc_weights[count] = encodedw
    return enc_weights
    
#Encoded weights back to decimals to look at differences
def encoded_to_decimal(enc_weights):
    back_weights = [0]*5
    count = -1
    for encw in enc_weights:
        count = count +1
        if encw > 64:
            encw = -(128 -encw)
        back_weights[count] = encw/64    
    return back_weights

#Performs a correction to encoded weights that don't sum to 256
def correction(enc_weights_not_corrected, back_weights):
    
    diff = 256 - sum(enc_weights_not_corrected)
    weightdiff = np.subtract(weights, back_weights)
    #print(weightdiff)
    
    if diff>0 and diff <6:
        pos = np.argsort(weightdiff)[-diff:]
        for k in pos:
            enc_weights_not_corrected[k] = enc_weights_not_corrected[k] + 1   
    elif diff< 0 and diff> -6:
        d = abs(diff)
        pos = np.argsort(weightdiff)[:d]
        
        #print(pos)
        for k in pos:
            enc_weights_not_corrected[k] = enc_weights_not_corrected[k] - 1   
                       
    else:

        enc_weights_not_corrected = 0
        print("\nError. The sum of the encoded weights differ from 256 by more than 5. Needs more investigation- dodgy numbers?")
        print("The encoded weights for this XTAL have been set to 0")
    
    return enc_weights_not_corrected
        

#-------------EEpos------------------------

#Need a list of all the different stripids
diff_strips = []
for i in data.index:
    w = data.loc[i, 'stripid'] 
    if w not in diff_strips:
        diff_strips.append(w)

#print(len(diff_strips))

#Loops over each stripid, calulating average weights and encoding them
for s in diff_strips:
    xtals = data.loc[(data['stripid']==s)]
    #print(xtals)
    
    weights = [xtals['w3'].mean(), xtals['w4'].mean(), xtals['w5'].mean(), xtals['w6'].mean(), xtals['w7'].mean()]
    
    #Encodes the decimal weights
    enc_weights_not_corrected = decimal_to_encoded(weights)
    
    #Calculates the decimal weights from the encoded ones - result will be one of the 1/64 allowable weights
    back_weights = encoded_to_decimal(enc_weights_not_corrected)
    
    #Check if sum =256, if not correction is performed on the weight(s) that are furthest from the weights they were originally describing
    if sum(enc_weights_not_corrected) != 256:
        enc_weights_corrected = correction(enc_weights_not_corrected, back_weights)
    else:
        enc_weights_corrected = enc_weights_not_corrected
        
    #need to set encws for each of the xtals in the strip to the correct values    
    if enc_weights_corrected != 0:
        
        for i in xtals.index:
            data.at[i, 'encw1'] = enc_weights_corrected[0]
            data.at[i, 'encw2'] = enc_weights_corrected[1]
            data.at[i, 'encw3'] = enc_weights_corrected[2]
            data.at[i, 'encw4'] = enc_weights_corrected[3]
            data.at[i, 'encw5'] = enc_weights_corrected[4]
        
        #print(enc_weights_corrected)

    
data.head(20)

data.to_csv("./DOF_P_Jun18_5_1_20Nov_with_stripID_encw_NEW.txt", sep='\t',  index=False, float_format='%g')

print("finished")



finished
